# Team E&Ez third place solution for "Autonomous Shopper Prediction by Cape AI"


By Geoffrey Frost, Kevin Eloff and Matthew Baas

We were inspired by SOTA skeleton-based action recognition techniques that rely on the inherent graph structure of human skeletons, whereby each joint is represented as a node, and bones as edges. For this competition, we implement variants of the [CTR-GCN architecture](https://arxiv.org/abs/2107.12213). This network relies on learnt and fixed graph topology (represented as adjacency matrices) to condition an aggregation of standard temporal convolutional layers. Unlike the base implementation which simply normalises and flattens features output by the last layer (to be fed into a linear classification), we leverage the temporal nature of these output features by:

1) A multi-head attention layer to weight features from specific instances in time more relevant to the action and, 
2) A transformer encoder network to produce a single context-rich feature vector.

Model 1's public test AUC was 0.878 (private 0.719)
Model 2's public test AUC was 0.853 (private 0.704)

The ensemble of these two models (and our final solution) resulted in a public test AUC of 0.881 and a private AUC of 0.712. Expected notebook runtime (training on a system with a GTX 1070ti, Ryzen 5 5600G and 16GB RAM) is 1-1.5 hours.

## Enviroment
We use Anconda `4.10.1` as our Python distribution. Please use the included `enviroment.yml` file to ensure all the correct packages and their respective versions are installed.

## Preprocessing
- Load train and test dataframes
- Extrapolate and interpolate between missing frames
- Split training data into train and validation sets
- Collate train, val and test into data dictionaries for later

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm, norm
from fastprogress.fastprogress import master_bar, progress_bar
from scipy import interpolate
import math
import pickle

In [2]:
train_df = pd.read_csv('Train.csv')
target_df = pd.read_csv('Train_Target.csv')
test_df = pd.read_csv('Test.csv')
train_df.shape, target_df.shape
train_df.head(3)

,Basket Masked,ID,noseX,noseY,leftEyeInnerX,leftEyeInnerY,leftEyeX,leftEyeY,leftEyeOuterX,leftEyeOuterY,...,rightAnkleX,rightAnkleY,leftHeelX,leftHeelY,rightHeelX,rightHeelY,leftFootIndexX,leftFootIndexY,rightFootIndexX,rightFootIndexY
0,Basket 1,ID_P1GM5F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Basket 1,ID_P1GM5F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Basket 1,ID_P1GM5F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
cols = train_df.columns
point_names = []
for col in cols[2::2]:
    point_names.append(col[:-1])
', '.join(point_names)

'nose, leftEyeInner, leftEye, leftEyeOuter, rightEyeInner, rightEye, rightEyeOuter, leftEar, rightEar, leftMouth, rightMouth, leftShoulder, rightShoulder, leftElbow, rightElbow, leftWrist, rightWrist, leftPinky, rightPinky, leftIndex, rightIndex, leftThumb, rightThumb, leftHip, rightHip, leftKnee, rightKnee, leftAnkle, rightAnkle, leftHeel, rightHeel, leftFootIndex, rightFootIndex'

In [4]:
REMOVE_STILLFRAMES = False
INTERPOLATE = True
EXTRAPOLATE = True
NORMALISE = False

In [5]:
def preprocess(df, is_test=False):
    baskets = []
    ids = []
    vals = []
    zero_idx = []
    
    for ID in progress_bar(df.ID.unique()[:]):
        point_data = df[df.ID == ID].copy().reset_index(drop=True)
        baskets.append(str(df[df.ID == ID]['Basket Masked'].iloc[0]))
        frames = []
        idx = []
        zeros = []
        
        for i,row in point_data.iterrows():
            frame = []
            for col in point_names:
                x = row.get(f'{col}X')
                y = row.get(f'{col}Y')
                if x == 0 and y == 0:
                    zeros.append(i)
                    break
                frame.append([x,y])

            if len(frame) > 0:
                frames.append(np.array(frame))
                idx.append(i)
                
        frames = np.array(frames)
        full_frames = np.zeros((30,len(point_names),2))
        
        if frames.shape[0] == 1:
            if REMOVE_STILLFRAMES and not is_test:
                continue
            elif EXTRAPOLATE:
                full_frames = np.array([frames[0].copy() for i in range(30)])
            else:
                full_frames[idx[0]] = frames[0]
        else:
            if INTERPOLATE:
                for i in range(full_frames.shape[1]):
#                     if len(idx)>3:
#                         interX = interpolate.interp1d(idx, frames[:,i,0], bounds_error=False, fill_value="extrapolate", kind='cubic')
#                         interY = interpolate.interp1d(idx, frames[:,i,1], bounds_error=False, fill_value="extrapolate", kind='cubic')
#                     else:
                    interX = interpolate.interp1d(idx, frames[:,i,0], bounds_error=False, fill_value="extrapolate", kind='linear')
                    interY = interpolate.interp1d(idx, frames[:,i,1], bounds_error=False, fill_value="extrapolate", kind='linear')
        
                    if EXTRAPOLATE:
                        for j in range(30):
                            full_frames[j,i] = np.array([interX(j), interY(j)])
                    else:
                        for j in range(idx[0], idx[-1]+1, 1):
                            full_frames[j,i] = np.array([interX(j), interY(j)])
            else:
                full_frames[idx] = frames
            
        vals.append(np.array(full_frames))
        ids.append(ID)
        zero_idx.append(zeros)
#         points[ID] = np.array(full_frames) 
                
#         if all_points.max() > 0:
#             m = np.abs(all_points).max(axis=1).max(axis=0)
#             a = all_points[:,[24,23],:]
#             hips = a[a!=0].reshape(int((a!=0).sum()/2),2).mean(axis=0)

#             points_list[ID] = np.array([(all_points[i]/m)-hips for i in range(30)])
    return baskets, ids, np.array(vals), np.array(zero_idx)

In [6]:
valid_pct = 0.18
np.random.seed(5)
idx = np.random.permutation(np.arange(len(train_df['Basket Masked'].unique())))
train_idx = idx[:int(len(train_df['Basket Masked'].unique())*(1-valid_pct))]
valid_idx = idx[int(len(train_df['Basket Masked'].unique())*(1-valid_pct)):]
train_baskets = train_df['Basket Masked'].unique()[train_idx]
valid_baskets = train_df['Basket Masked'].unique()[valid_idx]
print(f"Train baskets is of len: {len(train_baskets)} and validation baskets is of len: {len(valid_baskets)}")

Train baskets is of len: 88 and validation baskets is of len: 20


In [7]:
train_df_ = train_df.loc[train_df['Basket Masked'].isin(train_baskets)]
val_df_ = train_df.loc[train_df['Basket Masked'].isin(valid_baskets)]

In [8]:
train_baskets, train_ids, train_frames, train_zeros = preprocess(train_df_)
train_frames = train_frames.reshape(*train_frames.shape[:-2], -1)

<ipython-input-5-a99c1c97ed27>:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return baskets, ids, np.array(vals), np.array(zero_idx)


In [9]:
val_baskets, val_ids, val_frames, val_zeros = preprocess(val_df_)
val_frames = val_frames.reshape(*val_frames.shape[:-2], -1)

<ipython-input-5-a99c1c97ed27>:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return baskets, ids, np.array(vals), np.array(zero_idx)


In [10]:
test_baskets, test_ids, test_frames, test_zeros = preprocess(test_df)
test_frames = test_frames.reshape(*test_frames.shape[:-2], -1)

<ipython-input-5-a99c1c97ed27>:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return baskets, ids, np.array(vals), np.array(zero_idx)


In [11]:
print(f'train samples: {train_frames.shape}, validation samples: {val_frames.shape}, test samples: {test_frames.shape}')

train samples: (1077, 30, 66), validation samples: (265, 30, 66), test samples: (667, 30, 66)


In [12]:
# Creates a dataset dictionary 

def create_dataset(data, ids, zeros, target_df, test=False):

    dataset = {}
    dataset['inps'] = []

    if not test: dataset['tgts'] = []

    for i, id_ in enumerate(ids):
        frame_skips = np.zeros((30,))
        frame_skips[zeros[i]] = 1
        dataset['inps'].append(np.concatenate((data[i], np.expand_dims(frame_skips, axis=-1)), axis=1))

        if not test:
            label = int(target_df.iloc[target_df.index[target_df['ID'] == id_]]['Target'])
            
            if label != 1: label = 0
            dataset['tgts'].append(label)

    return dataset

In [13]:
train_dataset = create_dataset(train_frames, train_ids, train_zeros, target_df)
val_dataset = create_dataset(val_frames, val_ids, val_zeros, target_df)

In [14]:
with open('train_dataset_ex.pkl', 'wb') as handle:
    pickle.dump(train_dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('val_dataset_ex.pkl', 'wb') as handle:
    pickle.dump(val_dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
test_dataset = create_dataset(test_frames, test_ids, test_zeros, target_df, test=True)

with open('test_dataset_ex.pkl', 'wb') as handle:
    pickle.dump(test_dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Dataloader
- Take data dicts and turn them into dataloaders
- Max norm
- Simplify blaze pose joints

TODO: Add Figure here

In [16]:
import torch

torch.manual_seed(1234)

In [17]:
# p_7 = (p_8 + p_11) / 2
# p_14 = (p_15 + p_18) / 2

blaze_pose_mini_map = [(0,0), (1,1), (3,2), (7,3), (4,4), (6,5), (8,6), 
                       (12,8), (14,9), (16,10), (11, 11), (13,12), (15,13),
                       (24,15), (26,16), (28,17), (23,18), (25, 19), (27, 20)]

In [18]:
def mini_map(tmp1, tmp2):
    for m in blaze_pose_mini_map:
        tmp2[:,m[1]] = tmp1[:,m[0]]
    tmp2[:, 7] = (tmp2[:, 8] + tmp2[:, 11]) / 2
    tmp2[:, 14] =  (tmp2[:, 15] + tmp2[:, 18]) / 2
    return tmp2

In [19]:
MINI = True
BS = 64

In [20]:
with open('train_dataset_ex.pkl', "rb") as input_file:
    train_dataset = pickle.load(input_file)
    
with open('val_dataset_ex.pkl', "rb") as input_file:
    val_dataset = pickle.load(input_file)

for i, x in enumerate(train_dataset['inps']):
    x = x[:, :-1]
    if MINI:
        tmp1 = x.reshape((30, 33, 2))
        tmp2 = np.zeros((30, 21, 2))
        x = mini_map(tmp1, tmp2)
        train_dataset['inps'][i] = np.expand_dims(np.moveaxis(x, -1, 0), -1)
                                      
    else: train_dataset['inps'][i] = np.expand_dims(np.moveaxis(x.reshape((30, 33, 2)), -1, 0), -1)
            
max_0 = 0
max_1 = 0

for x in train_dataset['inps']:
    if max_0 < np.max(abs(x[0])): max_0 = np.max(abs(x[0]))
    if max_1 < np.max(abs(x[1])): max_1 = np.max(abs(x[1]))

for i, x in enumerate(train_dataset['inps']): train_dataset['inps'][i] = x / np.array([max_0, max_1])[:, np.newaxis, np.newaxis, np.newaxis]

for i, x in  enumerate(val_dataset['inps']):
    x = x[:, :-1]
    if MINI:
        tmp1 = x.reshape((30, 33, 2))
        tmp2 = np.zeros((30, 21, 2))
        x = mini_map(tmp1, tmp2)
        val_dataset['inps'][i] = np.expand_dims(np.moveaxis(x, -1, 0), -1) / np.array([max_0, max_1])[:, np.newaxis, np.newaxis, np.newaxis]

    else: val_dataset['inps'][i] = np.expand_dims(np.moveaxis(x.reshape((30, 33, 2)), -1, 0), -1) / np.array([max_0, max_1])[:, np.newaxis, np.newaxis, np.newaxis]

x = train_dataset['inps']
y = train_dataset['tgts']

dataset_train_ = []
for i, t in zip(x, y): dataset_train_.append([torch.tensor(i, dtype=torch.float), torch.tensor(t, dtype=torch.long)])

x = val_dataset['inps']
y = val_dataset['tgts']

dataset_val_ = []
for i, t in zip(x, y): dataset_val_.append([torch.tensor(i, dtype=torch.float), torch.tensor(t, dtype=torch.long)])

dataloader_train = torch.utils.data.DataLoader(
    dataset=dataset_train_,
    batch_size=BS,
    shuffle=True,
    num_workers=4)

dataloader_val = torch.utils.data.DataLoader(
    dataset=dataset_val_,
    batch_size=BS,
    shuffle=False,
    num_workers=4)

torch.save(dataloader_train, f'dataloader_train.pt')
torch.save(dataloader_val, f'dataloader_val.pt')

file = open("test_dataset_ex.pkl",'rb')
dataset_test = pickle.load(file)

for i, x in  enumerate(dataset_test['inps']):
    x = x[:, :-1]

    if MINI:
        tmp1 = x.reshape((30, 33, 2))
        tmp2 = np.zeros((30, 21, 2))
        x = mini_map(tmp1, tmp2)
        dataset_test['inps'][i] = np.expand_dims(np.moveaxis(x, -1, 0), -1) / np.array([max_0, max_1])[:, np.newaxis, np.newaxis, np.newaxis]
        
    else: dataset_test['inps'][i] = np.expand_dims(np.moveaxis(x.reshape((30, 33, 2)), -1, 0), -1) / np.array([max_0, max_1])[:, np.newaxis, np.newaxis, np.newaxis]

x = dataset_test['inps']

dataset_test_ = []
for i in x: dataset_test_.append(torch.tensor(i, dtype=torch.float))

dataloader_test = torch.utils.data.DataLoader(
    dataset=dataset_test_,
    batch_size=64,
    shuffle=False,
    num_workers=4)

torch.save(dataloader_test, f'dataloader_test.pt')

## Train and test
- Train model 1 (CTR-GCN w/ attention)
- Compute test preds
- Train model 2 (CTR-GCN w/ transformer)
- Compute test preds
- Ensamble

### Model 1

In [21]:
import torch
import torch.nn.functional as F
import pytorch_lightning as pl
from model_att import LitModel

In [22]:
import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

In [23]:
graph_args = {'strategy':'distance',
            'layout':'blazepose_mini'}

In [24]:
torch.manual_seed(1234)
torch.use_deterministic_algorithms(True) # Essential for reproducibility, but increases runtime substantially :c

model = LitModel(
                graph_args=graph_args,
                drop_out=0.0,
                learning_rate=1e-2)

dataloader_train = torch.load(f'dataloader_train.pt')
dataloader_val = torch.load(f'dataloader_val.pt')

checkpoint_callback = pl.callbacks.model_checkpoint.ModelCheckpoint(monitor='neg_val_auc',
                                                                filename='{epoch}-{val_loss:.2f}-{val_auc:.2f}',
                                                                save_on_train_epoch_end=False,
                                                                save_last=True)

learningrate_callback = pl.callbacks.LearningRateMonitor(logging_interval='step')

trainer = pl.Trainer(
            #auto_lr_find=True,
            callbacks=[checkpoint_callback,
                    learningrate_callback],
                    log_every_n_steps=4,
                    max_epochs=128,
                    check_val_every_n_epoch=1,
                    gpus=1)

pl.seed_everything(1234)

trainer.fit(model, 
        train_dataloaders=dataloader_train,
        val_dataloaders=dataloader_val)

Global seed set to 1234
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Global seed set to 1234
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Global seed set to 1234

   | Name      | Type               | Params
--------------------------------------------------
0  | data_bn   | BatchNorm1d        | 84    
1  | l1        | TCN_GCN_unit       | 9.9 K 
2  | l2        | TCN_GCN_unit       | 19.5 K
3  | l3        | TCN_GCN_unit       | 19.5 K
4  | l4        | TCN_GCN_unit       | 19.5 K
5  | l5        | TCN_GCN_unit       | 66.6 K
6  | l6        | TCN_GCN_unit       | 74.0 K
7  | l7        | TCN_GCN_unit       | 74.0 K
8  | l8        | TCN_GCN_unit       | 259 K 
9  | l9        | TCN_GCN_unit       | 290 K 
10 | l10       | TCN_GCN_unit       | 290 K 
11 | conv_norm | Conv2d             | 257   
12 | attention | MultiheadAttention | 1.8 K 
13 | fc        | Linear             | 44    
--------------------------------------------------

Validation sanity check: 0it [00:00, ?it/s]

/home/geoff/anaconda3/envs/masters/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Global seed set to 1234


Training: -1it [00:00, ?it/s]

/home/geoff/anaconda3/envs/masters/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:405: LightningDeprecationWarning: One of the returned values {'y_hat', 'y'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  warning_cache.deprecation(


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [25]:
dataloader_test = torch.load(f'dataloader_test.pt')
model.eval()
model.cuda()
preds = []

with torch.no_grad():
    for x in dataloader_test:
        x = x.cuda()
        preds.append(F.softmax(model(x), dim=-1)[:, -1])
        
y_hat = []
for batch in preds:
    y_hat += batch.detach().cpu().tolist()

In [26]:
ss = pd.read_csv('SampleSubmission.csv')

ss_copy = ss.copy()
for i, pred in enumerate(y_hat): 
    ss_copy.iloc[i, 1] = pred

ss_copy.to_csv('ctr-gcn-skip-mini-att.csv', index=False, sep=',')

### Model 2

In [28]:
from model_trans import LitModel

In [29]:
torch.manual_seed(1234)
torch.use_deterministic_algorithms(True)

model = LitModel(
                graph_args=graph_args,
                drop_out=0.0,
                learning_rate=1e-2)

dataloader_train = torch.load(f'dataloader_train.pt')
dataloader_val = torch.load(f'dataloader_val.pt')

checkpoint_callback = pl.callbacks.model_checkpoint.ModelCheckpoint(monitor='neg_val_auc',
                                                                filename='{epoch}-{val_loss:.2f}-{val_auc:.2f}',
                                                                save_on_train_epoch_end=False,
                                                                save_last=True)

learningrate_callback = pl.callbacks.LearningRateMonitor(logging_interval='step')

trainer = pl.Trainer(
            #auto_lr_find=True,
            callbacks=[checkpoint_callback,
                    learningrate_callback],
                    log_every_n_steps=4,
                    max_epochs=128,
                    check_val_every_n_epoch=1,
                    gpus=1)

pl.seed_everything(1234)

trainer.fit(model, 
        train_dataloaders=dataloader_train,
        val_dataloaders=dataloader_val)

Global seed set to 1234
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Global seed set to 1234
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Global seed set to 1234

   | Name        | Type               | Params
----------------------------------------------------
0  | data_bn     | BatchNorm1d        | 84    
1  | l1          | TCN_GCN_unit       | 9.9 K 
2  | l2          | TCN_GCN_unit       | 19.5 K
3  | l3          | TCN_GCN_unit       | 19.5 K
4  | l4          | TCN_GCN_unit       | 19.5 K
5  | l5          | TCN_GCN_unit       | 66.6 K
6  | l6          | TCN_GCN_unit       | 74.0 K
7  | l7          | TCN_GCN_unit       | 74.0 K
8  | l8          | TCN_GCN_unit       | 259 K 
9  | l9          | TCN_GCN_unit       | 290 K 
10 | l10         | TCN_GCN_unit       | 290 K 
11 | conv_norm   | Conv2d             | 257   
12 | upsample    | Linear             | 704   
13 | pos_encoder | PositionalEncoding | 0     
14 | transformer |

Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 1234


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [30]:
dataloader_test = torch.load(f'dataloader_test.pt')
model.eval()
model.cuda()
preds = []

with torch.no_grad():
    for x in dataloader_test:
        x = x.cuda()
        preds.append(F.softmax(model(x), dim=-1)[:, -1])
        
y_hat = []
for batch in preds:
    y_hat += batch.detach().cpu().tolist()

In [31]:
ss = pd.read_csv('SampleSubmission.csv')

ss_copy = ss.copy()
for i, pred in enumerate(y_hat): 
    ss_copy.iloc[i, 1] = pred

ss_copy.to_csv('ctr-gcn-skip-mini-trans.csv', index=False, sep=',')

### Ensamble

In [33]:
pred1 = pd.read_csv('ctr-gcn-skip-mini-att.csv')
pred2 = pd.read_csv('ctr-gcn-skip-mini-trans.csv')

pred1['Target'] = (pred1['Target'] + pred2['Target'])/2

pred1.to_csv('ensamble.csv', index=False, sep=',')

Done!